## Re-producing CNTK 104 Time Series Basics Solution Using a Number of Classifier models



### Define the imports

In [ ]:
import cntk as C
import cntk.tests.test_utils
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components

In [20]:
import os
import datetime
import pickle as  pkl
import numpy as np
import pandas as pd
import keras.models as km
import keras.layers as kl
import sklearn.preprocessing as spp
import sklearn.model_selection as smo
from sklearn.preprocessing import StandardScaler
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import (train_test_split, KFold)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

sns.set(style="whitegrid")
%matplotlib inline

### Install Alpha Vantage to source dialy financial data

- My Alpha Vintage key: E8AQO37EPBZULELG 

In [22]:
def installAlphaVantage():
    try:
        from  alpha_vantage.timeseries import TimeSeries
    except ImportError:
        !pip install alpha_vantage
        from  alpha_vantage.timeseries import TimeSeries

#installAlphaVantage()

In [21]:
# Set a random seed
np.random.seed(123)

### Get Data

In [25]:
data_cache_path = os.path.join("data", "Stock", "stock_SPY.pkl")

def getDailyDataFromAlphaVantage(data_cache_path, symbol='SPY'):
    try:
        ts = TimeSeries(key = "E8AQO37EPBZULELG", output_format='pandas' )
        data, meta = ts.get_daily(symbol=symbol, outputsize='full')
    except:
        raise Exception("Data could not be downloaded from Alpha Vantage") 
    dir = os.path.dirname(data_cache_path)
    if not os.path.exists(dir):
        os.makedirs(dir)

    if not os.path.isfile(data_cache_path):
        print("Caching data..", data_cache_path )
        with open(data_cache_path, 'wb') as f:
            pkl.dump(data, f, protocol = 2)
    return data

def downloadSPYDataForLast20Years():
    if os.path.exists(data_cache_path):
        print("File already exists", data_cache_path)
        data = pd.read_pickle(data_cache_path)
    else:
        print("Source the data from Alpha Vantage")
        data = getDailyDataFromAlphaVantage(data_cache_path)
    return data

def getSPYDataSimilarToCNTK104():
    spy_data = downloadSPYDataForLast20Years()
    spy_data_new = spy_data["2000-01-02":"2017-01-01"]
    return spy_data_new

spy_data = getSPYDataSimilarToCNTK104()
spy_data.head(5)

File already exists data/Stock/stock_SPY.pkl


,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2000-01-03,148.2500,148.2500,143.8750,145.4375,8164300.0
2000-01-04,143.5312,144.0625,139.6406,139.7500,8089800.0
2000-01-05,139.9375,141.5312,137.2500,140.0000,12177900.0
2000-01-06,139.6250,141.5000,137.7500,137.7500,6227200.0
2000-01-07,140.3125,145.7500,140.0625,145.7500,8066500.0
